In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error
from scipy.ndimage.interpolation import shift
from math import sin, cos, sqrt, atan2, radians 
from sklearn import tree, svm, linear_model, ensemble, neighbors, naive_bayes 
import dateutil
import os
from pathlib import Path
from joblib import dump, load
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import make_scorer
import ast


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
status_path = Path('../filtered_status.csv')
stations_path = Path('../station.csv')

In [3]:
status_df = pd.read_csv(status_path, parse_dates=['time'])
stations_df = pd.read_csv(stations_path)

In [4]:
status_df

,station_id,bikes_available,docks_available,time
0,2,2,25,2013-08-29 12:06:01
1,2,2,25,2013-08-29 12:07:01
2,2,2,25,2013-08-29 12:08:01
3,2,2,25,2013-08-29 12:09:01
4,2,2,25,2013-08-29 12:10:01
...,...,...,...,...
71977905,84,8,7,2015-08-31 23:55:02
71977906,84,8,7,2015-08-31 23:56:01
71977907,84,8,7,2015-08-31 23:57:02
71977908,84,8,7,2015-08-31 23:58:02


In [5]:
stations_df

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
...,...,...,...,...,...,...,...
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014


In [6]:
#il seguente vettore è stato ottenuto in analisi fatte precedentemente. 
SFancisco_stations = [39, 41, 42, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58, 59, 60, 
                      61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 82]

### Calcolo la distanza tra le diverse stazioni della città di San Francisco

In [7]:
#function to retrieve distance between 2 stations
def getDistance(lat_a, long_a, lat_b, long_b):
    # approximate radius of earth in km
    R = 6373.0    
    lat1=radians(lat_a)
    lat2=radians(lat_b)
    lon1=radians(long_a)
    lon2=radians(long_b)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [8]:
coordinates_stations_df = stations_df[stations_df["id"].isin(SFancisco_stations)]
coordinates_stations_df = coordinates_stations_df[["id", "lat", "long"]]
records = coordinates_stations_df.to_records(index=False)
list_coo = list(records)
# print(list_coo)

voc_distances={}
for i in range(len(list_coo)):
    for j in range(i+1,len(list_coo)):
        station1=list_coo[i][0]
        station2=list_coo[j][0]
        lat_i=float(list_coo[i][1])
        long_i=float(list_coo[i][2])
        lat_j=float(list_coo[j][1])
        long_j=float(list_coo[j][2])
        distance=getDistance(lat_i, long_i, lat_j, long_j)
        id_stations=str(station1)+' '+str(station2)
        voc_distances[id_stations]=distance


In [9]:
# voc_distances

In [10]:
distance_dictionary = {}


for station_id in SFancisco_stations:
    distance_dictionary[station_id] = []
    
    delta0_stations = []
    delta1_stations = []
    delta2_stations = []

    for key in voc_distances.keys():
        if str(station_id) in key:
            distance = voc_distances[key]

            if key.split(" ")[0] == str(station_id):
                other=key.split(" ")[1]
            elif key.split(" ")[1] == str(station_id):
                other=key.split(" ")[0]
            
            if distance<=1:
                delta0_stations.append(other)         
            elif distance<=2:
                delta1_stations.append(other)
            elif distance<=3:
                delta2_stations.append(other)
        
    distance_dictionary[station_id].append(list(delta0_stations))
    distance_dictionary[station_id].append(list(delta1_stations))
    distance_dictionary[station_id].append(list(delta2_stations))


In [ ]:
#Questo dizionario contiene come chiave tutte le stazioni e come valori corrisopondenti tre liste contenenti
# le stazioni che distantano dalla stazione della chiave rispettivamente <1 km, tra 1 e  

# distance_dictionary

In [9]:
# distance_dictionary[63]

[['41',
  '45',
  '47',
  '49',
  '50',
  '51',
  '39',
  '54',
  '55',
  '56',
  '57',
  '61',
  '62',
  '64',
  '68',
  '71',
  '74',
  '75',
  '76',
  '77'],
 ['42', '46', '48', '58', '59', '65', '69', '70', '72', '73', '82'],
 ['60', '66', '67']]

### Testo i pattern estratti

Adesso per ogni stazione devo:
- 1) andare a leggere il suo specifico test set
- 2) andare a leggere i pattern estratti dal training set
- 3) controllare se i pattern sono verificati 
- 3.1) per ogni entry del test set vedere se esiste almeno una regola verificata
- 3.2) se esiste la predizione sarà "QP" (QuasiPiena), altrimenti "N" (Normal)
- 4) costruire un vettore delle predizioni in base al punto precedente
- 5) calcolare un la matrice di confusione
- 6) ottenere dalla matrice di confusione le metriche

In [70]:
#Parametri considerati per la costruzione dei dataset della classificazione
time_interval = 30
window_width = 5

#Parametri usati per l'estazione dei pattern
space_interval = 1000

#Parametri usati per il filtraggio dei pattern
conf_threshold = 40 #valore intero (%)
sup_threshold = 0

#Flag utile a considerare soltanto i pattern contenenti esclusivamente QuasiPiena
filter_normal = True

#Numero di pattern da metchare
num_tot_pat = 45

In [ ]:
#Se considero soltanto i pattern contenenti soltanto QuasiPiena
if filter_normal:
    pattern_path = f"./Results_extraction/{time_interval}min_{space_interval}m/filtered_results({conf_threshold}%-{sup_threshold})_StateChange_almostFull_{time_interval}_{space_interval}_0(3-3)_ordered_by_confidence.txt"
    output_path = f'Test_results/{time_interval}min/results_almostFull_{num_tot_pat}match_{time_interval}min_{window_width}_({conf_threshold}%-{sup_threshold}).txt'
else:
    pattern_path = f"./Results_extraction/{time_interval}min_{space_interval}m/filtered_results({conf_threshold}%-{sup_threshold})_StateChange_Normal_almostFull_{time_interval}_{space_interval}_0(3-3)_ordered_by_confidence.txt"
    output_path = f'Test_results/{time_interval}min/results_Normal_almostFull_{num_tot_pat}match_{time_interval}min_{window_width}_({conf_threshold}%-{sup_threshold}).txt'
    
file = open(output_path, "w")
file.write(f'PATTERN TESING RESULTS:\n\n')
tot_fp = 0
tot_tp = 0
tot_fn = 0
tot_tn = 0


for station_id in SFancisco_stations:
    
    test_path = Path(f'../Traditional_classification/datasets/{time_interval}_{window_width}/station{station_id}_test.csv')
    test_df = pd.read_csv(test_path)
    

    y_test = test_df["status"]
    y_pred = []

    #itero le righe del test set
    for index, row in test_df.iterrows():
        num_row=0
        num_pat_matched = 0
        #print(row['docks_av_41_T4'], row['status'])
        prediction = "N"

        #apro il file ed itero su tutti i pattern
        with open(pattern_path, "r") as ifile:
            for line in ifile:
                num_row+=1
                #elimino gli header del file
                if "[[[" not in line:
                    continue

                flag = '' # sarà usaro per la condizione di uscita se una regola non metcha

                #Trasformo la linea testuale in una lista
                line_list = ast.literal_eval(line)
                
#                 #Prelevo il conseguente
#                 last = line_list[0][0][-1]
                
                #elimino il conseguente
                prior = line_list[0][0][:-1]

                #Prelevo l'ultimo elemento
                last = prior[-1]

                #Ricavo l'istante temporale dell'ultimo slot che corrisponderà all'istante 0 nella tabella del test set
                curr_time_slot = int(last[0].split('_')[1][1])

                #Itero gli slot di tempo presenti nell'antecedente
                for item in prior:
    #                 print(f"1_{flag}")

                    matched_neigh = 0

                    #controllo se è già stata settata la condizione di uscita
                    if flag == "next_line":
                        break

                    #ricavo la stringa contenuta nell'i-esimo slot e la splitto per ottenere la lista dei controlli da fare
                    check_list = item[0].split(",")

                    #itero sulla lista contenuta nello stesso slot temporale
                    for el in check_list:
    #                     print(f"2_{flag}")
                        #controllo se è già stato settata la condizione di uscita
                        if flag == "next_line":
                            break

                        el_status = el.split('_')[0]
                        el_time_slot = int(el.split('_')[1][1])
                        el_space_slot = int(el.split('_')[2])     

                        feature_time_slot = curr_time_slot - el_time_slot

                        #se la stazione è quella di riferimento
                        if el_space_slot == 0:

                            #A partire da queste info costruisco la feature da andare a controllare
                            feature_str = f"docks_av_{station_id}_T{feature_time_slot}"

    #                         print(feature_str)

                            #Se la regola non è verificata, passo alla prossima
                            if (el_status == "QuasiPiena" and row[feature_str] > 2) or (el_status == "Normal" and row[feature_str] <= 2):
                                flag = "next_line"
    #                             print(f"3_{flag}")
                                break


                        #se invece si tratta di una delle stazioni vicine:
                        else:
                            neighbors = distance_dictionary[station_id][el_space_slot-1]
                            for neigh_id in neighbors:
                                #inizialmente lo stesso al caso negativo
                                flag = "next_line"

                                #se il vicino corrente ha già verificato la regola, passo al vicino successivo 
                                # (utile nei casi in cui si hanno piu regole nello stesso delta spaziale)
                                if neigh_id == matched_neigh:
                                    continue

                                feature_str = f"docks_av_{neigh_id}_T{feature_time_slot}"
    #                             print(feature_str)
                            
                                #se il vicino corrente non rispetta il pattern, passo al vicino successivo
                                if (el_status == "QuasiPiena" and row[feature_str] > 2) or (el_status == "Normal" and row[feature_str] <= 2):
                                    continue

                                #se invece viene trovato un vicino che rispetta il pattern, rimetto il flag ='' ed esco dal ciclo  
                                flag=''
                                matched_neigh= neigh_id
                                break

                            if flag == "next_line":
                                break


                #se a questo punto il flag non è stato settato a "next_line", vuol dire che nessuna delle condizioni sfavorevoli 
                # precedenti è stata verificata, quindi il pattern ha metchato. 
                if flag != "next_line":
                    #incremento il numero di pattern verificati
                    num_pat_matched +=1
                    
                    #se il numero di pattern verificati è uguale al numero di pattern da verificare desiderato
                    if num_pat_matched == num_tot_pat: 
                        # Setto pred a "QP" ed esco
                        prediction = "QP"
        #                 print(f'PATTERN MATCHED NUM: {num_row}')
                        break

        y_pred.append(prediction)


    cm = confusion_matrix(y_test, y_pred)

    str_= f'PATTERN TEST FOR STATION {station_id}' + '\n'
    str_ += f'Confusion matrix:' + '\n'
    str_ += str(cm) + '\n'

    tn, fp, fn, tp = cm.ravel()
    str_+= f'tp={tp}, fn={fn}, fp={fp}, tn={tn}' +'\n'

    test_accuracy = (tn + tp) / (tn + fp + fn + tp)
    #test_recall = (tp) / (tp + fn)
    #test_precision = (tp) / (tp + fp)
    test_recall = recall_score(y_test, y_pred, pos_label='QP')
    test_precision = precision_score(y_test, y_pred, pos_label='QP', zero_division=0)
    test_f1_score = f1_score(y_test, y_pred, pos_label='QP')

    str_+= f'accuracy={test_accuracy}; recall={test_recall}; precision={test_precision}; f1_score= {test_f1_score}' +'\n\n'
    str_+= "-"*10 +'\n\n'

    tot_fp += fp
    tot_tp += tp
    tot_fn += fn
    tot_tn += tn

    file.write(str_)
    print(str_)

avg_accuracy = (tot_tn + tot_tp) / (tot_tn + tot_fp + tot_fn + tot_tp)
avg_recall = (tot_tp) / (tot_tp + tot_fn)
avg_precision = (tot_tp) / (tot_tp + tot_fp)
avg_f1_score = 2*(1/((1/avg_recall)+(1/avg_precision)))

avg_str = "AVERAGE VALUES FOR PATTERN TEST:\n"
avg_str += "Confusion matrix:\n"
avg_str += f"[[{tot_tn} {tot_fp}]\n[{tot_fn} {tot_tp}]]\n"
avg_str += f"tot_tp={tot_tp}, tot_fn={tot_fn}, tot_fp={tot_fp}, tot_tn={tot_tn}\n"
avg_str += f"accuracy={avg_accuracy}; recall={avg_recall}; precision={avg_precision}; f1_score={avg_f1_score}"
print(avg_str)
file.write(avg_str)
file.close()


In [ ]:
len(y_pred)

#### Tabella contenente tutti i risultati 

In [4]:

rows = []
result_matrix = []
columns = ["avg_recall", "avg_precision", "avg_f1_score"]

states = ["Normal_almostFull", "almostFull"]
confidences = [0, 55, 50, 40]
matches = [1, 2, 5, 10, 20, 30, 35, 40, 45]
time_interval = 60
num_interval = 5

for status in states:
    for conf in confidences:
        for match in matches:
                #the following path is to check the existence of the file
                path = Path(f"///home/bigdata-01QYD/s270240/bike_sharing/Associative_classifier/Test_results/{time_interval}min/results_{status}_{match}match_{time_interval}min_{num_interval}_({conf}%-0).txt")

                if path.is_file():
                    if status == "Normal_almostFull":
                        row = f"N/QP_conf{conf}%_{match}match"
                    else:
                        row = f"QP_conf{conf}%_{match}match"
                    
                    rows.append(row)
                    input_file_path = f"Test_results(1)/{time_interval}min/results_{status}_{match}match_{time_interval}min_{num_interval}_({conf}%-0).txt"

                
                    with open(input_file_path, "r") as file:
                        lines = file.read().splitlines()
                        last_line = lines[-1]
#                         print (last_line)
                        results = last_line.split(';')
                        result_values = []
                        for result in results:
                            value = round(float(result.split('=')[1]), 4)
                            result_values.append(value)
                        result_matrix.append(result_values[1:])

results_table = pd.DataFrame(result_matrix, columns=columns, index=pd.Index(rows))
print(results_table.to_latex(bold_rows=True))
results_table


\begin{tabular}{lrrr}
\toprule
{} &  avg\_recall &  avg\_precision &  avg\_f1\_score \\
\midrule
\textbf{N/QP\_conf50\%\_2match } &      0.6827 &         0.6813 &        0.6820 \\
\textbf{N/QP\_conf50\%\_5match } &      0.6827 &         0.6813 &        0.6820 \\
\textbf{N/QP\_conf50\%\_10match} &      0.6827 &         0.6813 &        0.6820 \\
\textbf{QP\_conf40\%\_5match   } &      0.3968 &         0.7734 &        0.5245 \\
\textbf{QP\_conf40\%\_10match  } &      0.2455 &         0.8077 &        0.3766 \\
\textbf{QP\_conf40\%\_20match  } &      0.1151 &         0.8571 &        0.2029 \\
\textbf{QP\_conf40\%\_30match  } &      0.0386 &         0.8680 &        0.0739 \\
\textbf{QP\_conf40\%\_35match  } &      0.0291 &         0.8958 &        0.0564 \\
\textbf{QP\_conf40\%\_40match  } &      0.0117 &         0.8966 &        0.0232 \\
\textbf{QP\_conf40\%\_45match  } &      0.0000 &         0.0000 &        0.0000 \\
\bottomrule
\end{tabular}



,avg_recall,avg_precision,avg_f1_score
N/QP_conf50%_2match,0.6827,0.6813,0.6820
N/QP_conf50%_5match,0.6827,0.6813,0.6820
N/QP_conf50%_10match,0.6827,0.6813,0.6820
QP_conf40%_5match,0.3968,0.7734,0.5245
QP_conf40%_10match,0.2455,0.8077,0.3766
QP_conf40%_20match,0.1151,0.8571,0.2029
QP_conf40%_30match,0.0386,0.8680,0.0739
QP_conf40%_35match,0.0291,0.8958,0.0564
QP_conf40%_40match,0.0117,0.8966,0.0232
QP_conf40%_45match,0.0000,0.0000,0.0000
